# Day 12: Hot Springs

In [1]:
import re
from functools import cache

In [2]:
def parseInput(filename):
    records = []
    with open(filename) as f:
        for line in f:
            row,groups = line.strip().split(' ')
            groups = tuple([int(x) for x in groups.split(',')])
            records.append((row,groups))
    return records

In [3]:
records=parseInput('../testInputs/day12.txt')
records=parseInput('../inputs/day12.txt')

## Part 1

In [4]:
def testRow(row,groups):
    indexFirstQ = row.find('?')
    #no more question marks in string - evaluate if arrangement is allowed
    if indexFirstQ<0:
        brokenGroups = re.findall('#+',row)
        #check if we have same number of groupings
        if len(brokenGroups) != len(groups):
            return 0
        #check if lenght of each group of broken springs matches
        for i in range(len(brokenGroups)):
            if len(brokenGroups[i]) != groups[i]:
                return 0
        #print(row)
        return 1
    #there are still question marks in string - replace first question mark found once with '.' and once with '#' and recurse
    else:
        nArrangements = 0
        newRow = row[:indexFirstQ]+ '.' + row[(indexFirstQ+1):]
        nArrangements += testRow(newRow,groups)
        
        newRow = row[:indexFirstQ]+ '#' + row[(indexFirstQ+1):]
        nArrangements += testRow(newRow,groups)
        
        return nArrangements
    
def testRecords(records):
    nArrangements = 0
    
    for row,groups in records:
        dummy = testRow(row,groups)
        nArrangements += dummy
        #print(dummy)
        #print('')
    return nArrangements

In [5]:
testRecords(records)

7169

## Part 2

So... Part 1 brute force is not going to work here...

We need a new plan: 
- check character by character, passing only the remaining string along
- keep track of how long the current group is
- use functools.cache
- conditions of replacing a ? with .
    - we have no current open group
    - our current group has reached its end
- conditions of replacing a ? with a #
    - we have no current open group but can open one
    - the current open group has not reached its end
- fail conditions when encountering a .
    - the current open group has not reached its end
- fail conditions when encountering a #
    - the current open group has already reached its end
    - we have no currend open group and there are no more groups left
- fail conditions when string is empty
    - we have an open group that terminates prematurely
    - we have no open group but still groups left
- success conditions:
    - the remaining string is empty and we have no open groups and no groups left

In [6]:
@cache
def testRowPart2(rowRemaining,groupsRemaining, lenCurrentOpenGroup):
    
    # we have no chars left
    if rowRemaining == '':
        # we have no groups left or last remaining group has ended: success
        if len(groupsRemaining) == 0 or (len(groupsRemaining) == 1 and groupsRemaining[0] == lenCurrentOpenGroup):
            return 1
        #we still have groups left and/or last group terminated prematurely: fail
        else:
            return 0
    
    nextChar = rowRemaining[0]
    nArrangements = 0
    
    if nextChar == '.':
        #we currently have no open group
        if lenCurrentOpenGroup == 0:
            nArrangements += testRowPart2(rowRemaining[1:], groupsRemaining,0)
        #we have an open group and terminated it correctly
        elif lenCurrentOpenGroup == groupsRemaining[0]:
            nArrangements += testRowPart2(rowRemaining[1:], groupsRemaining[1:],0)
        #we have an open group and terminated it prematurely -> no possible matching arrangements from this point forward
        else:
            return 0
    
    elif nextChar == '#':
        # we currently have no open group but still possible groups left or
        # we currently have an open group which has not reached its full length
        if len(groupsRemaining)>0 and groupsRemaining[0]>lenCurrentOpenGroup:
            nArrangements += testRowPart2(rowRemaining[1:], groupsRemaining,lenCurrentOpenGroup+1)
        # we have have no possible groups left or the current group is already full -> no possible matching arrangements
        else:
            return 0
    
    #next char is '?'
    else:
        # if we have no open group, we can always place a .
        # if we have groups left, we can also place a #
        if lenCurrentOpenGroup == 0:
            nArrangements += testRowPart2(rowRemaining[1:], groupsRemaining,0)
            if len(groupsRemaining) > 0:
                nArrangements += testRowPart2(rowRemaining[1:], groupsRemaining,lenCurrentOpenGroup+1)
        # if we have an open group, we can place a # if the group has not reached full length
        # and a . if it has and needs to terminate at this point
        else:
            if lenCurrentOpenGroup < groupsRemaining[0]:
                nArrangements += testRowPart2(rowRemaining[1:], groupsRemaining,lenCurrentOpenGroup+1)
            else:
                nArrangements += testRowPart2(rowRemaining[1:], groupsRemaining[1:],0)
                
    return nArrangements   
        
def testRecordsPart2(records):
    nArrangements = 0
    
    unfoldedRecords = [('?'.join([row]*5), groups*5) for row,groups in records]
    
    for row,groups in unfoldedRecords:
        dummy = testRowPart2(row,groups,0)
        nArrangements += dummy
        #print(dummy)
    return nArrangements

In [7]:
testRecordsPart2(records)

1738259948652